# Transformer from scratch — Blank Notebook (Aing)

이 노트북은 `transformer_from_scratch.py`를 기반으로 **Transformer 핵심 구성요소**(Self-Attention / Multi-Head / Add&Norm / FFN / Encoder-Decoder / Mask)를  
`____` **빈칸을 채우며** 논문 **Eq.(1), Eq.(2), Fig.2**의 흐름을 코드로 연결하는 실습 자료입니다.

- 빈칸은 **이해에 핵심인 지점만** 뚫었습니다. (의미 없는 빈칸 X)
- 각 변수명은 논문 표기와 매핑되도록 구성했습니다. (예: `Q,K,V`, `attention_logits_QK`, `attention_weights`)
- 텐서 차원이 바뀌는 연산 직후에는 **반드시 Shape 주석**으로 검증하게 되어 있습니다.

> 사용법  
> 1) 위에서부터 내려오며 `____`만 채우기  
> 2) Shape 주석과 동일한지 수시로 확인하기  
> 3) 마지막 `__main__` 테스트로 end-to-end 동작 확인하기


## (선택) 실행을 위한 설치

이미 환경에 PyTorch/Jupyter가 있다면 건너뛰세요.


In [ ]:
# 필수: PyTorch (텐서 연산/모델 정의)
!pip install torch

## 학습 목표 (초급→중급)

1. **Scaled Dot-Product Attention (Eq.1)** 을 코드로 구현하고, `QK^T / sqrt(d_k)`의 의미를 설명할 수 있다.  
2. **Multi-Head Attention**에서 `(N, seq, d_model)` → `(N, heads, seq, d_k)`로의 변환을 직접 구현할 수 있다.  
3. **Add & Norm + FFN (Eq.2)** 블록을 residual 관점에서 설명하고 구현할 수 있다.  
4. **Encoder / Decoder의 3가지 Attention**(self / cross / masked self)을 코드 흐름으로 구분할 수 있다.  
5. **Source mask / Target(causal) mask**가 왜 필요한지, 어디에 적용되는지 코드로 설명할 수 있다.


## (헷갈림 방지) CheatSheet ↔ 노트북 매핑

- **Eq.(1) Attention(Q,K,V)** ↔ `SelfAttention.forward`의  
  `attention_logits_QK` → `attention_weights` → `attention_out`
- **Multi-Head** ↔ `values/keys/queries`를 head로 쪼개는 `reshape/permute` + 마지막 `W_O` projection
- **Eq.(2) FFN** ↔ `TransformerBlock.forward`의 `feed_forward_out`
- **Fig.2 Encoder-Decoder** ↔ `Encoder`, `DecoderBlock`, `Decoder`, `Transformer.forward`
- **Mask** ↔ `make_src_mask`, `make_trg_mask` 그리고 attention logits에 적용되는 `masked_fill`


In [ ]:
"""
A from scratch implementation of Transformer network,
following the paper Attention is all you need with a
few minor differences. I tried to make it as clear as
possible to understand and also went through the code
on my youtube channel!
"""

import torch
import torch.nn as nn


## 1) Scaled Dot-Product Attention (Eq.1) ↔ `SelfAttention.forward`
## 2) Multi-Head Attention (MHA) ↔ split → attention → concat → projection

- **핵심 수식(Eq.1)**:  \( \text{softmax}(QK^T/\sqrt{d_k})V \)
- `attention_logits_QK`는 head별 `(query, key)` 점수표입니다.
- `attention_weights`는 softmax 이후의 확률이며, 마지막에 `V`를 가중합해 `attention_out`을 만듭니다.

### 사고 질문
- (why) 내적 점수에 `1/\sqrt{d_k}` 스케일링을 넣는 이유는?  
- (how) head로 쪼개면 표현력이 왜 늘어날까? (단일 head와 비교)


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.d_model = embed_size
        self.h = heads
        # [CookBook step.2] d_k = d_model // h ; assert d_k * h == d_model
        self.d_k = self.d_model // ________
        assert (
            ____________ * ____________ == ____________
        ), "Embedding size needs to be divisible by heads"

        # [CookBook step.6] Q = XW^Q, K = XW^K, V = XW^V (Eq.1)
        self.W_V = nn.Linear(__________, ____________)
        self.W_K = nn.Linear(__________, ____________)
        self.W_Q = nn.Linear(__________, ____________)
        # [CookBook step.12] W^O output projection after Concat(heads) (Fig.2)
        self.W_O = nn.Linear(__________, ____________)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = ____________

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # [CookBook step.6] Linear projections to Q, K, V
        V = self.W_V(values)  # (N, value_len, d_model)
        K = self.W_K(keys)  # (N, key_len, d_model)
        Q = self.W_Q(query)  # (N, query_len, d_model)

        # [CookBook step.7] Split into heads: (N, L, d_model) -> (N, L, h, d_k)
        V = V.reshape(N, value_len, ____________, ____________)  # (N, value_len, h, d_k)
        K = K.reshape(N, key_len, ____________, ____________)  # (N, key_len, h, d_k)
        Q = Q.reshape(N, query_len, ____________, ____________)  # (N, query_len, h, d_k)

        # [CookBook step.8] energy = QK^T (Eq.1)
        attention_logits_QK = torch.einsum("____________________", [Q, K])
        # (N, h, query_len, key_len)

        # [CookBook step.9] mask == 0 -> -inf
        if mask is not None:
            attention_logits_QK = attention_logits_QK.masked_fill(__________ == 0, float("-1e20"))
            # (N, h, query_len, key_len)

        # [CookBook step.10] softmax( energy / sqrt(d_k), dim=key_len_axis ) (Eq.1)
        attention_weights = torch.softmax(attention_logits_QK / (__________ ** (1 / 2)), dim=____)
        # (N, h, query_len, key_len)

        # [CookBook step.11] out = Attention(...)V (Eq.1)
        out_heads = torch.einsum("____________________", [attention_weights, V])
        # (N, query_len, h, d_k)
        out = out_heads.reshape(N, query_len, ____________ * ____________)
        # (N, query_len, d_model)

        # [CookBook step.12] Concat(heads) W^O
        out = self.W_O(out)
        # (N, query_len, d_model)

        return out


## 5) Add & Norm + FFN (Eq.2) ↔ `TransformerBlock.forward`

- **Residual(Add)**: 원본 입력을 보존한 채, 변환 결과를 더합니다.
- **LayerNorm(Norm)**: 분포를 안정화합니다.
- **FFN(Eq.2)**:  \( \text{FFN}(x)=\max(0, xW_1+b_1)W_2+b_2 \)

### 사고 질문
- (why) Attention 뒤에 바로 FFN을 한 번 더 넣는 이유는?  
- (how) residual이 없으면 역전파에서 어떤 문제가 생길까?


In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        # [CookBook step.14] FFN(x)=max(0, xW1+b1)W2+b2 (Eq.2)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        # [CookBook step.13] Multi-Head Attention sub-layer
        multihead_attention_output = self.attention(value, key, query, mask)
        # (N, query_len, d_model)

        # [CookBook step.13] Add & Norm: LayerNorm(x + Sublayer(x))
        attention_residual_add = ____________ + ____________
        # (N, query_len, d_model)
        post_attention_layernorm = self.norm1(attention_residual_add)
        # (N, query_len, d_model)
        x = self.dropout(post_attention_layernorm)
        # (N, query_len, d_model)

        # [CookBook step.14] FFN in explicit steps (so shapes are visible)
        ffn_linear1_output = self.feed_forward[0](x)
        # (N, query_len, forward_expansion*d_model)
        ffn_relu_output = self.feed_forward[1](ffn_linear1_output)
        # (N, query_len, forward_expansion*d_model)
        ffn_linear2_output = self.feed_forward[2](ffn_relu_output)
        # (N, query_len, d_model)

        # [CookBook step.14] Add & Norm after FFN
        ffn_residual_add = ____________ + ____________
        # (N, query_len, d_model)
        post_ffn_layernorm = self.norm2(ffn_residual_add)
        # (N, query_len, d_model)
        out = self.dropout(post_ffn_layernorm)
        # (N, query_len, d_model)

        return out


## 6) Embedding + Positional Encoding ↔ `Encoder/Decoder.forward`

- 입력 토큰을 `word_embedding`으로 **(N, seq)** → **(N, seq, d_model)** 로 바꿉니다.
- `position_embedding`을 더해 **순서 정보**를 주입합니다.
- 이후 dropout을 거쳐 Block stack으로 들어갑니다.

### 사고 질문
- (why) 순서 정보가 없으면 Self-Attention은 어떤 문제가 생길까?  
- (how) 학습형 position embedding과 sinusoidal의 장단점은?


In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, src_token_ids, src_padding_mask):
        # [CookBook step.1] src_token_ids: (N, src_len)
        N, src_len = src_token_ids.shape

        # [CookBook step.5] positions = arange(0, L).expand(N, L)
        position_ids = torch.arange(0, ____________).to(self.device)
        # (src_len,)
        positions = position_ids.expand(__________, ____________)
        # (N, src_len)

        # [CookBook step.4~5] token embedding + position embedding
        token_embedding_d_model = self.word_embedding(src_token_ids)
        # (N, src_len, d_model)
        positional_embedding_d_model = self.position_embedding(positions)
        # (N, src_len, d_model)
        out = self.dropout(____________ + ____________)
        # (N, src_len, d_model)

        # [CookBook step.15] Encoder stack: out = layer(out, out, out, src_mask)
        for layer in self.layers:
            out = layer(____________, ____________, ____________, ____________)
            # (N, src_len, d_model)

        return out


## 3) Encoder / Decoder의 3가지 Attention 매핑 ↔ `DecoderBlock.forward`

DecoderBlock에는 보통 2개의 attention이 있습니다.

1. **Masked Self-Attention**: Decoder 내부에서 미래 토큰을 못 보게 causal mask 적용  
2. **Cross-Attention(Encoder-Decoder Attention)**: Query는 decoder, Key/Value는 encoder output  
3. (Encoder는) **Self-Attention**만 사용

### 사고 질문
- (why) decoder self-attention에는 반드시 causal mask가 필요할까?  
- (how) cross-attention에서 Q/K/V의 출처를 코드로 정확히 짚어보자.


In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    #   - Decoder Masked Self-Attention
    #   - Encoder-Decoder Attention (Cross-Attention)
    def forward(self, x, value, key, src_mask, trg_mask):
        # [CookBook step.16] Masked self-attention (future tokens blocked by trg_mask)
        masked_self_attention_output = self.attention(____________, ____________, ____________, ____________)
        # (N, trg_len, d_model)

        # [CookBook step.16] Add & Norm
        residual_add = ____________ + ____________
        # (N, trg_len, d_model)
        query = self.dropout(self.norm(residual_add))
        # (N, trg_len, d_model)

        # [CookBook step.17] Cross-attention + FFN via TransformerBlock(value=enc_out, key=enc_out, query=query)
        out = self.transformer_block(____________, ____________, ____________, ____________)
        # (N, trg_len, d_model)

        return out


## 6) Embedding + Positional Encoding ↔ `Encoder/Decoder.forward`

- 입력 토큰을 `word_embedding`으로 **(N, seq)** → **(N, seq, d_model)** 로 바꿉니다.
- `position_embedding`을 더해 **순서 정보**를 주입합니다.
- 이후 dropout을 거쳐 Block stack으로 들어갑니다.

### 사고 질문
- (why) 순서 정보가 없으면 Self-Attention은 어떤 문제가 생길까?  
- (how) 학습형 position embedding과 sinusoidal의 장단점은?


In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg_token_ids, enc_out, src_padding_mask, trg_causal_mask):
        # [CookBook step.1] trg_token_ids: (N, trg_len)
        N, trg_len = trg_token_ids.shape

        # [CookBook step.5] positions = arange(0, L).expand(N, L)
        position_ids = torch.arange(0, ____________).to(self.device)
        # (trg_len,)
        positions = position_ids.expand(__________, ____________)
        # (N, trg_len)

        # [CookBook step.4~5] token embedding + position embedding
        token_embedding_d_model = self.word_embedding(trg_token_ids)
        # (N, trg_len, d_model)
        positional_embedding_d_model = self.position_embedding(positions)
        # (N, trg_len, d_model)
        x = self.dropout(____________ + ____________)
        # (N, trg_len, d_model)

        # [CookBook step.18] Decoder stack
        for layer in self.layers:
            x = layer(____________, ____________, ____________, ____________, ____________)
            # (N, trg_len, d_model)

        # [CookBook step.18] vocab projection -> logits
        out = self.fc_out(____________)
        # (N, trg_len, trg_vocab_size)

        return out


## 4) Mask(마스크) ↔ `make_src_mask`, `make_trg_mask`

- **Source mask**: PAD 토큰을 attention에서 무시하기 위함 (`src_pad_idx`)
- **Target mask(causal)**: 미래 토큰을 보지 못하게 하는 **상삼각 마스크**

적용 위치: attention logits에 `masked_fill(mask==0, -1e20)` 처럼 큰 음수로 가려 softmax 후 0이 되게 합니다.

### 사고 질문
- (why) logits 단계에서 마스킹해야 softmax 후 정확히 0이 될까?  
- (how) target mask의 shape를 head/배치 차원까지 맞추는 흐름을 추적해보자.


In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src_token_ids):
        # [CookBook step.3] src_padding_mask = (src != pad).unsqueeze(1).unsqueeze(2)
        src_is_not_pad = (src_token_ids != ____________)
        # (N, src_len)
        src_padding_mask = src_is_not_pad.unsqueeze(____)
        # (N, 1, src_len)
        src_padding_mask = src_padding_mask.unsqueeze(____)
        # (N, 1, 1, src_len)
        return src_padding_mask.to(self.device)

    def make_trg_mask(self, trg_token_ids):
        # [CookBook step.3] trg_causal_mask = tril(ones(L,L)).expand(N, 1, L, L)
        N, trg_len = trg_token_ids.shape
        trg_ones = torch.ones((__________, ____________))
        # (trg_len, trg_len)
        trg_lower_triangular = torch.tril(trg_ones)
        # (trg_len, trg_len)
        trg_causal_mask = trg_lower_triangular.expand(__________, 1, ____________, ____________)
        # (N, 1, trg_len, trg_len)

        return trg_causal_mask.to(self.device)

    def forward(self, src_token_ids, trg_token_ids):
        # [CookBook step.19] src_mask -> encoder -> decoder
        src_padding_mask = self.make_src_mask(____________)
        # (N, 1, 1, src_len)
        trg_causal_mask = self.make_trg_mask(____________)
        # (N, 1, trg_len, trg_len)

        enc_src = self.encoder(____________, ____________)
        # (N, src_len, d_model)
        out = self.decoder(____________, ____________, ____________, ____________)
        # (N, trg_len, trg_vocab_size)

        return out


In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    )
    # (N, src_len)
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)
    # (N, trg_len)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        device=device,
    ).to(device)

    trg_input_ids = trg[:, :-1]
    # (N, trg_len-1)
    out = model(x, trg_input_ids)
    # (N, trg_len-1, trg_vocab_size)
    print(out.shape)


### ✅ Check 7: (데이터 다운로드 없이) 더미 배치로 **1 step 학습 루프** 스모크 테스트

아래 코드는 **데이터셋 없이도** Transformer 학습 루프가 돌아가는지 확인하는 최소 테스트입니다.

- **Teacher Forcing 시프트 패턴(표준):**  
  `trg_input = trg[:, :-1]` / `trg_y = trg[:, 1:]`
- **CrossEntropy reshape 패턴(표준):**  
  `logits: (N, T, V) -> (N*T, V)` / `trg_y: (N, T) -> (N*T)`
- 위 빈칸(`SelfAttention/Block/Encoder/Decoder/...`)을 다 채운 뒤 실행하세요.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

device = "cuda" if torch.cuda.is_available() else "cpu"

# --- tiny toy vocab ---
src_vocab_size = 50
trg_vocab_size = 60
src_pad_idx = 0
trg_pad_idx = 0

# model hyperparams (paper-base 느낌, but tiny for smoke test)
model = Transformer(
    src_vocab_size=src_vocab_size,
    trg_vocab_size=trg_vocab_size,
    src_pad_idx=src_pad_idx,
    trg_pad_idx=trg_pad_idx,
    embed_size=128,
    num_layers=2,
    forward_expansion=4,
    heads=4,
    dropout=0.1,
    device=device,
    max_length=64,
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# --- dummy batch (variable lengths + padding) ---
src_batch = [
    torch.tensor([1, 5, 6, 4, 3, 9, 2]),
    torch.tensor([1, 8, 7, 3, 4, 5]),
]
trg_batch = [
    torch.tensor([1, 7, 4, 3, 5, 9, 2, 2]),
    torch.tensor([1, 5, 6, 2, 4, 7]),
]

src = pad_sequence(src_batch, batch_first=True, padding_value=src_pad_idx).to(device)
# (N, src_len)
trg = pad_sequence(trg_batch, batch_first=True, padding_value=trg_pad_idx).to(device)
# (N, trg_len)

trg_input = trg[:, :-1]
# (N, trg_len-1)
trg_y = trg[:, 1:]
# (N, trg_len-1)

logits = model(src, trg_input)
# (N, trg_len-1, trg_vocab_size)

logits_flat = logits.reshape(-1, logits.size(-1))
# (N*(trg_len-1), trg_vocab_size)
trg_y_flat = trg_y.reshape(-1)
# (N*(trg_len-1),)

optimizer.zero_grad()
loss = criterion(logits_flat, trg_y_flat)
loss.backward()
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
optimizer.step()

print("✅ smoke loss:", float(loss.item()))


## 8) 실습: Multi30k 학습을 **실제로** 돌려보기 (논문 설정 정렬)

아래 코드는 `seq2seq_transformer.py`의 흐름을 가져오되, **이 노트북의 Transformer 구현(Encoder–Decoder, Fig.2)** 을 그대로 사용하도록 재구성한  
**Multi30k(De→En) 학습 스켈레톤**입니다.

- 이 섹션은 **빈칸이 없습니다.**  
  (단, **위의 빈칸 구현이 완료되어야** 실행됩니다.)
- Multi30k는 논문에서 사용한 WMT14보다 훨씬 작은 데이터라 **논문 BLEU를 그대로 기대하면 안 됩니다.**
- 그래도 아래 “훈련 레시피”는 가능한 한 논문 설명에 맞춰 정렬했습니다.

### 논문 정렬 포인트(훈련 레시피)
- Optimizer: **Adam(β1=0.9, β2=0.98, ε=1e-9)**
- Learning rate schedule: **Noam warmup(4000) + inverse sqrt decay**
- Regularization: **dropout=0.1**, **label smoothing=0.1**(PAD는 ignore)

### 이 노트북 구현 ↔ 논문 차이(중요)
- (Paper) **Sinusoidal Positional Encoding** vs (Here) **Learned positional embedding**
- (Paper) **BPE/word-piece** vs (Here) **spaCy word tokenizer**
- (Paper) **weight tying**(임베딩/출력 가중치 공유) vs (Here) 미적용

### 필요한 패키지(처음 1회)
```bash
pip install datasets sacrebleu spacy
python -m spacy download de_core_news_sm
python -m spacy download en_core_web_sm
```

> ✅ 처음에는 `num_epochs=1`로 “동작 확인”만 하고, 그 다음 epochs를 늘리세요.


In [ ]:
# train_multi30k.py (paper-aligned-ish) — notebook friendly
# - 기반: seq2seq_transformer.py
# - 변경: (1) torchtext legacy 대신 HF datasets 사용 (Multi30k 다운로드/로딩 안정)
#        (2) 이 노트북의 Transformer(from scratch) 그대로 사용
#        (3) 논문 훈련 레시피(Adam betas/eps, Noam LR, label smoothing) 반영

import math
import random
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import spacy

# --- optional: BLEU (sacrebleu) ---
try:
    import sacrebleu
except Exception:
    sacrebleu = None

# --- HF datasets for Multi30k ---
try:
    from datasets import load_dataset
except Exception:
    load_dataset = None


SPECIAL_TOKENS = {
    "pad": "<pad>",
    "unk": "<unk>",
    "sos": "<sos>",
    "eos": "<eos>",
}


def set_seed(seed: int = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def build_vocab(sentences, tokenize_fn, max_size: int = 10000, min_freq: int = 2):
    """Word-level vocab builder (toy / study-friendly)."""
    counter = Counter()
    for s in sentences:
        counter.update(tokenize_fn(s))

    # reserve specials at the beginning so indices are stable
    itos = [
        SPECIAL_TOKENS["pad"],
        SPECIAL_TOKENS["unk"],
        SPECIAL_TOKENS["sos"],
        SPECIAL_TOKENS["eos"],
    ]

    for tok, freq in counter.most_common():
        if freq < min_freq:
            continue
        if tok in itos:
            continue
        itos.append(tok)
        if len(itos) >= max_size:
            break

    stoi = {tok: i for i, tok in enumerate(itos)}
    return stoi, itos


class Multi30kWordDataset(Dataset):
    """(de, en) sentence pairs -> (src_ids, trg_ids)"""

    def __init__(
        self,
        split,
        src_tokenize_fn,
        trg_tokenize_fn,
        src_stoi,
        trg_stoi,
        src_max_len: int = 100,
        trg_max_len: int = 100,
    ):
        self.split = split
        self.src_tokenize_fn = src_tokenize_fn
        self.trg_tokenize_fn = trg_tokenize_fn
        self.src_stoi = src_stoi
        self.trg_stoi = trg_stoi
        self.src_max_len = src_max_len
        self.trg_max_len = trg_max_len

        self.src_unk = src_stoi[SPECIAL_TOKENS["unk"]]
        self.trg_unk = trg_stoi[SPECIAL_TOKENS["unk"]]
        self.src_sos = src_stoi[SPECIAL_TOKENS["sos"]]
        self.src_eos = src_stoi[SPECIAL_TOKENS["eos"]]
        self.trg_sos = trg_stoi[SPECIAL_TOKENS["sos"]]
        self.trg_eos = trg_stoi[SPECIAL_TOKENS["eos"]]

    def __len__(self):
        return len(self.split)

    def __getitem__(self, idx):
        example = self.split[idx]
        src_text = example["de"]
        trg_text = example["en"]

        src_tokens = self.src_tokenize_fn(src_text)[: self.src_max_len - 2]
        trg_tokens = self.trg_tokenize_fn(trg_text)[: self.trg_max_len - 2]

        src_ids = [self.src_sos] + [self.src_stoi.get(t, self.src_unk) for t in src_tokens] + [
            self.src_eos
        ]
        trg_ids = [self.trg_sos] + [self.trg_stoi.get(t, self.trg_unk) for t in trg_tokens] + [
            self.trg_eos
        ]

        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(trg_ids, dtype=torch.long)


def make_collate_fn(src_pad_idx: int, trg_pad_idx: int):
    def collate_fn(batch):
        src_list = [b[0] for b in batch]
        trg_list = [b[1] for b in batch]
        src = pad_sequence(src_list, batch_first=True, padding_value=src_pad_idx)
        # (N, src_len)
        trg = pad_sequence(trg_list, batch_first=True, padding_value=trg_pad_idx)
        # (N, trg_len)
        return src, trg

    return collate_fn


def noam_lr_lambda(step: int, d_model: int, warmup_steps: int = 4000):
    """Paper: lr = d_model^{-0.5} * min(step^{-0.5}, step * warmup^{-1.5})"""
    step = max(step, 1)
    return (d_model ** -0.5) * min(step ** -0.5, step * (warmup_steps ** -1.5))


@torch.no_grad()
def greedy_decode(
    model,
    src_ids_1d: torch.Tensor,
    src_pad_idx: int,
    trg_sos_idx: int,
    trg_eos_idx: int,
    max_len: int,
    device: str,
):
    """Greedy decoding for quick sanity check (not beam search)."""
    model.eval()

    src = src_ids_1d.unsqueeze(0).to(device)
    # (N=1, src_len)

    generated = [trg_sos_idx]

    for _ in range(max_len):
        trg = torch.tensor(generated, dtype=torch.long, device=device).unsqueeze(0)
        # (N=1, trg_len)

        logits = model(src, trg)
        # (N=1, trg_len, trg_vocab_size)

        next_token = int(logits[0, -1].argmax(dim=-1).item())
        generated.append(next_token)

        if next_token == trg_eos_idx:
            break

    return generated


def train_one_epoch(model, loader, optimizer, scheduler, criterion, device: str):
    model.train()
    total_loss = 0.0

    for src, trg in loader:
        src = src.to(device)
        # (N, src_len)
        trg = trg.to(device)
        # (N, trg_len)

        # --- Teacher forcing shift (표준 패턴) ---
        trg_input = trg[:, :-1]
        # (N, trg_len-1)
        trg_y = trg[:, 1:]
        # (N, trg_len-1)

        logits = model(src, trg_input)
        # (N, trg_len-1, trg_vocab_size)

        logits_flat = logits.reshape(-1, logits.size(-1))
        # (N*(trg_len-1), trg_vocab_size)
        trg_y_flat = trg_y.reshape(-1)
        # (N*(trg_len-1),)

        optimizer.zero_grad()
        loss = criterion(logits_flat, trg_y_flat)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += float(loss.item())

    return total_loss / max(1, len(loader))


@torch.no_grad()
def evaluate(model, loader, criterion, device: str):
    model.eval()
    total_loss = 0.0

    for src, trg in loader:
        src = src.to(device)
        trg = trg.to(device)

        trg_input = trg[:, :-1]
        trg_y = trg[:, 1:]

        logits = model(src, trg_input)

        logits_flat = logits.reshape(-1, logits.size(-1))
        trg_y_flat = trg_y.reshape(-1)

        loss = criterion(logits_flat, trg_y_flat)
        total_loss += float(loss.item())

    return total_loss / max(1, len(loader))


def main(
    num_epochs: int = 1,
    batch_size: int = 64,
    max_vocab_size: int = 10000,
    min_freq: int = 2,
    max_len: int = 100,
    d_model: int = 512,
    num_layers: int = 6,
    num_heads: int = 8,
    forward_expansion: int = 4,
    dropout: float = 0.1,
    warmup_steps: int = 4000,
):
    if load_dataset is None:
        raise ImportError("❌ datasets가 없습니다. 먼저 `pip install datasets`를 실행하세요.")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    set_seed(42)

    # --- Load Multi30k (bentrevett subset on HF hub) ---
    raw = load_dataset("bentrevett/multi30k")
    train_raw = raw["train"]
    valid_raw = raw["validation"]
    test_raw = raw["test"]

    # --- Tokenizers (spaCy) ---
    # spaCy v3+에서는 'de'/'en' shortcut이 아니라 full model name이 필요합니다.
    spacy_de = spacy.load("de_core_news_sm")
    spacy_en = spacy.load("en_core_web_sm")

    def tokenize_de(text: str):
        return [tok.text.lower() for tok in spacy_de.tokenizer(text)]

    def tokenize_en(text: str):
        return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

    # --- Build vocab (word-level) ---
    src_stoi, src_itos = build_vocab(train_raw["de"], tokenize_de, max_size=max_vocab_size, min_freq=min_freq)
    trg_stoi, trg_itos = build_vocab(train_raw["en"], tokenize_en, max_size=max_vocab_size, min_freq=min_freq)

    src_pad_idx = src_stoi[SPECIAL_TOKENS["pad"]]
    trg_pad_idx = trg_stoi[SPECIAL_TOKENS["pad"]]
    trg_sos_idx = trg_stoi[SPECIAL_TOKENS["sos"]]
    trg_eos_idx = trg_stoi[SPECIAL_TOKENS["eos"]]

    # --- Datasets / Loaders ---
    train_ds = Multi30kWordDataset(train_raw, tokenize_de, tokenize_en, src_stoi, trg_stoi, max_len, max_len)
    valid_ds = Multi30kWordDataset(valid_raw, tokenize_de, tokenize_en, src_stoi, trg_stoi, max_len, max_len)
    test_ds = Multi30kWordDataset(test_raw, tokenize_de, tokenize_en, src_stoi, trg_stoi, max_len, max_len)

    collate_fn = make_collate_fn(src_pad_idx, trg_pad_idx)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # --- Model (this notebook's from-scratch Transformer) ---
    model = Transformer(
        src_vocab_size=len(src_itos),
        trg_vocab_size=len(trg_itos),
        src_pad_idx=src_pad_idx,
        trg_pad_idx=trg_pad_idx,
        embed_size=d_model,
        num_layers=num_layers,
        forward_expansion=forward_expansion,
        heads=num_heads,
        dropout=dropout,
        device=device,
        max_length=max_len,
    ).to(device)

    # --- Paper-aligned optimizer + schedule ---
    optimizer = optim.Adam(model.parameters(), lr=1.0, betas=(0.9, 0.98), eps=1e-9)

    scheduler = optim.lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=lambda step: noam_lr_lambda(step + 1, d_model=d_model, warmup_steps=warmup_steps),
    )

    criterion = nn.CrossEntropyLoss(
        ignore_index=trg_pad_idx,
        label_smoothing=0.1,  # paper: ε_ls = 0.1
    )

    # --- Train ---
    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion, device)
        valid_loss = evaluate(model, valid_loader, criterion, device)

        print(f"epoch={epoch:02d} train_loss={train_loss:.4f} valid_loss={valid_loss:.4f}")

        # quick qualitative check: translate a random validation sample
        sample = valid_raw[random.randrange(len(valid_raw))]
        src_text = sample["de"]
        trg_text = sample["en"]

        src_tokens = tokenize_de(src_text)[: max_len - 2]
        src_ids = [src_stoi[SPECIAL_TOKENS["sos"]]] + [src_stoi.get(t, src_stoi[SPECIAL_TOKENS["unk"]]) for t in src_tokens] + [src_stoi[SPECIAL_TOKENS["eos"]]]
        src_ids = torch.tensor(src_ids, dtype=torch.long)

        pred_ids = greedy_decode(
            model=model,
            src_ids_1d=src_ids,
            src_pad_idx=src_pad_idx,
            trg_sos_idx=trg_sos_idx,
            trg_eos_idx=trg_eos_idx,
            max_len=50,
            device=device,
        )

        pred_tokens = [trg_itos[i] for i in pred_ids]
        pred_tokens = [t for t in pred_tokens if t not in {SPECIAL_TOKENS["sos"], SPECIAL_TOKENS["eos"], SPECIAL_TOKENS["pad"]}]

        print("DE:", src_text)
        print("GT:", trg_text)
        print("PR:", " ".join(pred_tokens))
        print("-" * 80)

    # --- BLEU (optional) ---
    if sacrebleu is None:
        print("ℹ️ sacrebleu가 없어서 BLEU를 생략합니다. (pip install sacrebleu)")
        return

    # quick BLEU on a small subset (speed)
    model.eval()
    preds = []
    refs = []

    for i in range(200):
        ex = test_raw[i]
        src_text = ex["de"]
        ref_text = ex["en"]

        src_tokens = tokenize_de(src_text)[: max_len - 2]
        src_ids = [src_stoi[SPECIAL_TOKENS["sos"]]] + [src_stoi.get(t, src_stoi[SPECIAL_TOKENS["unk"]]) for t in src_tokens] + [src_stoi[SPECIAL_TOKENS["eos"]]]
        src_ids = torch.tensor(src_ids, dtype=torch.long)

        pred_ids = greedy_decode(
            model=model,
            src_ids_1d=src_ids,
            src_pad_idx=src_pad_idx,
            trg_sos_idx=trg_sos_idx,
            trg_eos_idx=trg_eos_idx,
            max_len=50,
            device=device,
        )

        pred_tokens = [trg_itos[i] for i in pred_ids]
        pred_tokens = [t for t in pred_tokens if t not in {SPECIAL_TOKENS["sos"], SPECIAL_TOKENS["eos"], SPECIAL_TOKENS["pad"]}]
        preds.append(" ".join(pred_tokens))
        refs.append(ref_text)

    bleu = sacrebleu.corpus_bleu(preds, [refs]).score
    print(f"BLEU (greedy, first 200 test samples) = {bleu:.2f}")


# ✅ 실행 예시 (처음엔 epochs를 줄여서!)
# main(num_epochs=1)


## (정답 공개) — 정말 마지막에만 확인하세요

In [ ]:
# ===========================
# ANSWERS — 빈칸에 들어갈 내용만 (마지막에 확인하세요)
# ===========================

# --- SelfAttention ---
# [__init__]
# self.d_k = self.d_model // self.h
# self.d_k * self.h == self.d_model
# self.W_V = nn.Linear(self.d_model, self.d_model)
# self.W_K = nn.Linear(self.d_model, self.d_model)
# self.W_Q = nn.Linear(self.d_model, self.d_model)
# self.W_O = nn.Linear(self.d_model, self.d_model)

# [forward]
# N = query.shape[0]
# V = V.reshape(N, value_len, self.h, self.d_k)  # (N, value_len, h, d_k)
# K = K.reshape(N, key_len, self.h, self.d_k)  # (N, key_len, h, d_k)
# Q = Q.reshape(N, query_len, self.h, self.d_k)  # (N, query_len, h, d_k)
# attention_logits_QK = torch.einsum("nqhd,nkhd->nhqk", [Q, K])
# attention_logits_QK = attention_logits_QK.masked_fill(mask == 0, float("-1e20"))
# attention_weights = torch.softmax(attention_logits_QK / (self.d_k ** (1 / 2)), dim=3)
# out_heads = torch.einsum("nhql,nlhd->nqhd", [attention_weights, V])
# out = out_heads.reshape(N, query_len, self.h * self.d_k)


# --- TransformerBlock ---
# [forward]
# attention_residual_add = multihead_attention_output + query
# ffn_residual_add = ffn_linear2_output + x


# --- Encoder ---
# [forward]
# position_ids = torch.arange(0, src_len).to(self.device)
# positions = position_ids.expand(N, src_len)
# out = self.dropout(token_embedding_d_model + positional_embedding_d_model)
# out = layer(out, out, out, src_padding_mask)


# --- DecoderBlock ---
# [forward]
# masked_self_attention_output = self.attention(x, x, x, trg_mask)
# residual_add = masked_self_attention_output + x
# out = self.transformer_block(value, key, query, src_mask)


# --- Decoder ---
# [forward]
# position_ids = torch.arange(0, trg_len).to(self.device)
# positions = position_ids.expand(N, trg_len)
# x = self.dropout(token_embedding_d_model + positional_embedding_d_model)
# x = layer(x, enc_out, enc_out, src_padding_mask, trg_causal_mask)
# out = self.fc_out(x)


# --- Transformer ---
# [make_src_mask]
# src_is_not_pad = (src_token_ids != self.src_pad_idx)
# src_padding_mask = src_is_not_pad.unsqueeze(1)
# src_padding_mask = src_padding_mask.unsqueeze(2)

# [make_trg_mask]
# trg_ones = torch.ones((trg_len, trg_len))
# trg_causal_mask = trg_lower_triangular.expand(N, 1, trg_len, trg_len)

# [forward]
# src_padding_mask = self.make_src_mask(src_token_ids)
# trg_causal_mask = self.make_trg_mask(trg_token_ids)
# enc_src = self.encoder(src_token_ids, src_padding_mask)
# out = self.decoder(trg_token_ids, enc_src, src_padding_mask, trg_causal_mask)
